In [2]:
import sys
#import h2o
#from h2o.estimators.glm import H2OGeneralizedLinearEstimator
#from h2o.estimators.gbm import H2OGradientBoostingEstimator
#from h2o.estimators.random_forest import H2ORandomForestEstimator
#from h2o.estimators.deeplearning import H2ODeepLearningEstimator
#from h2o.h2o import _locate
#import time
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
#import matplotlib
#import matplotlib.pyplot as plt

#%matplotlib inline
#h2o.init(max_mem_size_GB=8)

In [3]:
#!/usr/bin/python

'''
Based on https://www.kaggle.com/justdoit/rossmann-store-sales/xgboost-in-python-with-rmspe/code
Public Score :  0.11389
Private Validation Score :  0.096959
'''
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-0.985) ** 2))
#0.985 & 1 
# RMSPE2=1/2[((x−909)/909)**2+((x−1100)/1100)**2]
# https://www.kaggle.com/c/rossmann-store-sales/forums/t/17601/correcting-log-sales-prediction-for-rmspe/99643#post99643
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

# Gather some features
def build_features(features, data):
    # remove NaNs
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
    # Use some properties directly
    features.extend(['Store', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
                     'CompetitionOpenSinceYear', 'Promo', 'Promo2', 'Promo2SinceWeek',
                     'Promo2SinceYear'])

    # add some more with a bit of preprocessing
    features.append('SchoolHoliday')
    data['SchoolHoliday'] = data['SchoolHoliday'].astype(float)

    features.extend(['StoreType', 'Assortment', 'StateHoliday'])
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)

    features.extend(['DayOfWeek', 'month', 'day', 'year'])
    data['year'] = data.Date.dt.year
    data['month'] = data.Date.dt.month
    data['day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek

## Start of main script

In [4]:
print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/train_filled_gap.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/store_prep.csv",sep=';')

print("Assume store open, if not provided")
train.fillna(1, inplace=True)
test.fillna(1, inplace=True)

print("Consider only open stores for training. Closed stores wont count into the score.")
train = train[train["Open"] != 0]
print("Use only Sales bigger then zero. Simplifies calculation of rmspe")
train = train[train["Sales"] > 0]

# Testinfo aus Kaggle, nicht für XGBoost da schon enthalten
train['Sales'] = np.log1p(train['Sales'])

print("Join with store")
train = train.merge(store, on='Store')
test = test.merge(store, on='Store')
print train.columns

features = []

print("augment features")
build_features(features, train)
build_features([], test)
print(features)


train.drop(['PromoInterval','Date','logSales','Customers'],axis=1,inplace=True)
train.to_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/train__filled_gap_prep.csv",index=False)
             
test.drop(['PromoInterval','Date'],axis=1,inplace=True)
test.to_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/test_prep.csv",index=False)

print train.columns
print train.shape

print test.columns
print test.shape
print('training data processed')

Load the training, test and store data using pandas
Assume store open, if not provided
Consider only open stores for training. Closed stores wont count into the score.
Use only Sales bigger then zero. Simplifies calculation of rmspe
Join with store
Index([u'Store', u'DayOfWeek', u'Date', u'Sales', u'Customers', u'Open',
       u'Promo', u'StateHoliday', u'SchoolHoliday', u'logSales', u'StoreType',
       u'Assortment', u'CompetitionDistance', u'CompetitionOpenSinceMonth',
       u'CompetitionOpenSinceYear', u'Promo2', u'Promo2SinceWeek',
       u'Promo2SinceYear', u'PromoInterval', u'PromoInt0', u'PromoInt1',
       u'PromoInt2', u'PromoInt3', u'PromoInt4'],
      dtype='object')
augment features
['Store', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'SchoolHoliday', 'StoreType', 'Assortment', 'StateHoliday', 'DayOfWeek', 'month', 'day', 'year']
Index([u'Store', u'DayOfWeek', u'Sales', u'Open', 

In [5]:
train.shape

(872238, 23)

## XGB Boost Train Model

## XGBoost Standard

In [ ]:
print("Train a XGBoost model")
                
params = {"objective": "reg:linear",
            "booster" : "gbtree",
            "eta":0.858086,
            "max_depth":11,
            "subsample":0.963407,
            "colsample_bytree":0.817634,
            "lambda":0.756323,
            "alpha":0.205266,
            "min_child_weight":7,
            "num_boost_round":9
         }
    
#params = {"objective": "reg:linear",
#          "booster" : "gbtree",
#          "eta": 0.09,
#          "max_depth": 10,
#          "subsample": 0.9,
#          "colsample_bytree": 0.7,
#          "silent": 1,
#            "seed": 1301
#          }

num_boost_round = 2500


X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)
y_train = X_train.Sales
y_valid = X_valid.Sales
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)

print("Validating")
yhat = gbm.predict(xgb.DMatrix(X_valid[features]))
error = rmspe(X_valid.Sales.values, np.expm1(yhat))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
# Make Submission
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv('xgboost_STANDARD.csv', index=False)

## XGBoost GridSearch
XGB1 * 0.16 + XGB2*0.39 + XGB3 *0.39 + RF * 0.06

In [6]:
features = [u'Store', u'DayOfWeek', u'Open', u'Promo', u'StateHoliday',
       u'SchoolHoliday', u'StoreType', u'Assortment', u'CompetitionDistance',
       u'CompetitionOpenSinceMonth', u'CompetitionOpenSinceYear', u'Promo2',
       u'Promo2SinceWeek', u'Promo2SinceYear', u'PromoInt0', u'PromoInt1',
       u'PromoInt2', u'PromoInt3', u'PromoInt4', u'year', u'month', u'day']

In [7]:
num_boost_round = 2500


X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)

y_train = X_train.Sales
y_valid = X_valid.Sales

#features = train.columns
#features.remove('Sales')

dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)


for eta in [0.009]:
    for max_depth in [130]:
        for subsample in [0.5]:
            for colsample_bytree in [0.5]:
                print("Train a XGBoost model", eta, max_depth, subsample, colsample_bytree)


#    "reg:linear" --linear regression
#    "reg:logistic" --logistic regression
#    "binary:logistic" --logistic regression for binary classification, output probability
#   "binary:logitraw" --logistic regression for binary classification, output score before logistic transformation
#    "count:poisson" --poisson regression for count data, output mean of poisson distribution
#        max_delta_step is set to 0.7 by default in poisson regression (used to safeguard optimization)
#    "multi:softmax" --set XGBoost to do multiclass classification using the softmax objective, you also need to set num_class(number of classes)
#    "multi:softprob" --same as softmax, but output a vector of ndata * nclass, which can be further reshaped to ndata, nclass matrix. The result contains predicted probability of each data point belonging to each class.
#   "rank:pairwise" --set XGBoost to do ranking task by minimizing the pairwise loss

                params = {"objective": "reg:linear",
                          "booster" : "gblinear", #getree, gblinear
                          "eta": eta,
                          "max_depth": max_depth,
                          "subsample": subsample,
                          "colsample_bytree": colsample_bytree,
                          "silent": 1,
                              "seed": 1301
                          }


                watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
                gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
                  early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)

                print("Validating")
                yhat = gbm.predict(xgb.DMatrix(X_valid[features]))
                error = rmspe(X_valid.Sales.values, np.expm1(yhat))
                print('RMSPE: {:.6f}'.format(error))

                print("Make predictions on the test set")
                dtest = xgb.DMatrix(test[features])
                test_probs = gbm.predict(dtest)
                # Make Submission
                result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
                result.to_csv('xgboost_gblinear_RMSPE985_'+str(error)+'_'+str(eta)+'_'+str(max_depth)+'_'+str(subsample)+'_'+str(colsample_bytree)+'.csv', index=False)
            del gbm, result, test_probs, dtest, yhat, error
            print 'Delete....'

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-rmspe:0.984436	eval-rmspe:0.984437
[1]	train-rmspe:0.983390	eval-rmspe:0.983396
[2]	train-rmspe:0.981175	eval-rmspe:0.981197
[3]	train-rmspe:0.976886	eval-rmspe:0.976942
[4]	train-rmspe:0.969293	eval-rmspe:0.969419
[5]	train-rmspe:0.956999	eval-rmspe:0.957250
[6]	train-rmspe:0.938749	eval-rmspe:0.939198
[7]	train-rmspe:0.914191	eval-rmspe:0.914922
[8]	train-rmspe:0.884766	eval-rmspe:0.885850
[9]	train-rmspe:0.854811	eval-rmspe:0.856275
[10]	train-rmspe:0.832662	eval-rmspe:0.834439
[11]	train-rmspe:0.829966	eval-rmspe:0.831852
[12]	train-rmspe:0.859190	eval-rmspe:0.860868
[13]	train-rmspe:0.928305	eval-rmspe:0.929455
[14]	train-rmspe:1.037280	eval-rmspe:1.037690
[15]	train-rmspe:1.179229	eval-rmspe:1.178854
[16]	train-rmspe:1.343934	eval-rmspe:1.342825
[17]	train-rmspe:1.522453	eval-rmspe:1.520700
[18]	train-rmspe:1.706053	eval-rmspe:1.703779
[19]	train-rmspe:1.888708	eval-rmspe:1.886038
[20]	train-rmspe:2.064656	eval

('Train a XGBoost model', 0.009, 130, 0.5, 0.5)
Validating
RMSPE: 1795.732371
Make predictions on the test set
Delete....


[111]	train-rmspe:2.294072	eval-rmspe:2.247134
Stopping. Best iteration:
[11]	train-rmspe:0.829966	eval-rmspe:0.831852



## Blend XGBoost
XGB1 * 0.16 + XGB2*0.39 + XGB3 *0.39 + RF * 0.06

In [5]:
eins = pd.read_csv('/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/xgboost_0.0864089444975_0.009_15_0.7_0.7.csv')
zwei = pd.read_csv('/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/xgboost_0.0868335094218_0.009_15_0.5_0.7.csv')
drei = pd.read_csv('/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/xgboost_0.0868709590517_0.009_15_0.7_1.csv')
vier = pd.read_csv('/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/xgboost_0.0868830056844_0.009_15_0.5_1.csv')
fuenf = pd.read_csv('/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/xgboost_0.0884385627912_0.009_15_0.5_0.5.csv')

In [17]:
blend = pd.DataFrame()
blend['Id'] = eins.Id
blend['Sales'] = 0.10*eins.Sales + 0.10*zwei.Sales + 0.10*drei.Sales + 0.10*vier.Sales + 0.60*fuenf.Sales
blend.to_csv('blend_xgboost.csv', index=False)

# XGB feature importances

In [7]:
# XGB feature importances

create_feature_map(features)
importance = gbm.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

featp = df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
fig_featp = featp.get_figure()
#fig_featp.savefig('feature_importance_xgb.png', bbox_inches='tight', pad_inches=1)

NameError: name 'gbm' is not defined

## Populating XGboost

In [ ]:
X_train, X_valid = train_test_split(train, test_size=0.012)
y_train = X_train.Sales
y_valid = X_valid.Sales
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)


##############################Evolutionary search #########################
'''
Data preprocessing is done. Now we are in place for some parameter optimization. 
Evolutionary Algorithm is a randomized meta optimization procedure that mimics natural evolution.
The algorithm proceeds with first creating an initial random population of
parameter values. The instances are scored using xgboost 5-fold CV. 
Next, a new generation of population  is created as follows:
- A small proportion of elite (i.e. top scoring) individuals is carried forward directly to the new population
- The rest of the population is filled with randomly created individuals, by:
    +randomly picking two parents from the top performing individuals of the last population (e.g. top 50%)
    +combine the 'genes' (parameter values) randomly to create a new individual that inherits 
    50% of the genes from each parent.
    +with a small probability, we mutate some gene's value
    
- The new population is evaluated, and the loop continues until convergence, or until 
a predefined number of generations has been reached. 
'''

from random import randint
import random

popSize=5; #population size, set from 20 to 100
eliteSize=0.1; #percentage of elite instances to be ratained 

paramList=['depth','nRound','eta','gamma','min_child_weight','lamda','alpha','colsample_bytree','subsample','fitness']

#Creating an initial population
population=pd.DataFrame(np.zeros(shape=(popSize,len(paramList))),columns = paramList);
population.depth=[randint(6,15) for p in range(0,popSize)]
#population.nRound=[randint(50,500) for p in range(0,popSize)]  #number of boosting round
population.nRound=[randint(5,10) for p in range(0,popSize)] #quick test
population.eta=[random.uniform(0.6, 1) for p in range(0,popSize)]
population.gamma=[random.uniform(0.01, 0.03) for p in range(0,popSize)]
population.min_child_weight=[randint(1,20) for p in range(0,popSize)]
population.lamda =[random.uniform(0.1,1) for p in range(0,popSize)]
population.alpha =[random.uniform(0.1, 1) for p in range(0,popSize)]
population.colsample_bytree=[random.uniform(0.7, 1) for p in range(0,popSize)]
population.subsample=[random.uniform(0.7, 1) for p in range(0,popSize)]
population.fitness=[random.uniform(100, 100) for p in range(0,popSize)]

#Creating a new population based on an existing one
def createNewPopulation(population,eliteSize=0.1,mutation_rate=0.2):
    population.sort(['fitness'],ascending=1,inplace=True)
    population.reset_index(drop=True,inplace=True)
    popSize=population.shape[0]
    nElite=int(round(eliteSize*popSize))
    
    new_population=population.copy(deep=True);
    for i in range(nElite,popSize):    #form a new population from the top 50% instances
        #get two random parents
        p1=randint(nElite,int(popSize/2))
        p2=randint(nElite,int(popSize/2))
        
        for attr in list(new_population.columns.values):
            #print attr, population[attr][i]
            if(random.uniform(0,1)>0.5 ):
                new_population.ix[i,attr]=population.ix[p1,attr]
            else:
                new_population.ix[i,attr]=population.ix[p2,attr]

            #injecting some mutation
            if(random.uniform(0,1)<mutation_rate ):
                attr=list(new_population.columns.values)[randint(0,8)]
                if(attr=='depth'):
                    new_population.ix[i,attr]= max(3,new_population.ix[i,attr]+randint(-2,2))
                elif(attr=='nRound'):
                    new_population.ix[i,attr]= max(10,new_population.ix[i,attr]+randint(-50,50))
                elif(attr=='eta'):
                    new_population.ix[i,attr]= max(0.1,new_population.ix[i,attr]+random.uniform(-0.05,0.05))
                elif(attr=='gamma'):
                    new_population.ix[i,attr]= max(0.1,new_population.ix[i,attr]+random.uniform(-0.005,0.005))
                elif(attr=='min_child_weight'):
                    new_population.ix[i,attr]= max(0,new_population.ix[i,attr]+randint(-2,2)  )                  
                elif(attr=='lamda'):
                    new_population.ix[i,attr]= max(0.1,new_population.ix[i,attr]+random.uniform(-0.05,0.05))                   
                elif(attr=='alpha'):
                    new_population.ix[i,attr]= max(0.1,new_population.ix[i,attr]+random.uniform(-0.05,0.05))                   
                elif(attr=='colsample_bytree'):
                    new_population.ix[i,attr]= max(0.6,new_population.ix[i,attr]+random.uniform(-0.05,0.05)) 
                elif(attr=='subsample'):
                    new_population.ix[i,attr]= max(0.6,new_population.ix[i,attr]+random.uniform(-0.05,0.05))                      
    return new_population

#score each instance using 5-fold CV
def testInstance(population,i,dtrain):
    params = {"objective": "reg:linear",
          "eta": population.eta[i],
          "max_depth": population.depth[i],
          "subsample": population.subsample[i],
          "colsample_bytree": population.colsample_bytree[i],
          "num_boost_round":int(population.nRound[i]),
          "lambda":population.lamda[i],
          "alpha":population.alpha[i],
          "gamma":population.gamma[i],
          "min_child_weight":population.min_child_weight[i],
          "silent": 1,
          #"seed": 1301
          } 
    history = xgb.cv(
        params,
        dtrain,  
        #early_stopping_rounds=30, #no early stopping in Python yet!!!
        num_boost_round  =int(population.nRound[i]),
        nfold=5, # number of CV folds
        nthread=-1, # number of CPU threads  
        show_progress=False,
        feval=rmspe_xg, # custom evaluation metric
        obj=RMSPE_objective
        #maximize=0 # the lower the evaluation score the better
        )
    return history["test-rmspe-mean"].iget(-1)

def printResult(filename,population,i,generation): #print best instances to file
    f1=open(filename, 'a')
    f1.write('Generation %d Best fitness %f\n' % (generation , population.fitness[i]))
    f1.write('"eta":%f\n'%population.eta[i])    
    f1.write('"max_depth":%f\n'%population.depth[i])    
    f1.write('"subsample":%f\n'%population.subsample[i])    
    f1.write('"colsample_bytree":%f\n'%population.colsample_bytree[i])    
    f1.write('"lambda":%f\n'%population.lamda[i])    
    f1.write('"alpha":%f\n'%population.alpha[i])    
    f1.write('"min_child_weight":%f\n'%population.min_child_weight[i])    
    f1.write('"num_boost_round":%f\n'%population.nRound[i])  
    f1.close()
           
#Main loop of the Evolutionary Algorithm: 
#Populations are created and avaluated.

nGeneration=2; #number of generations
for run in range(nGeneration):
    print("Generation %d\n" %run)
    population=createNewPopulation(population,eliteSize=0.1,mutation_rate=0.2)
    for i in range(popSize):
        print ("Testing instance %d "%i)
        population.ix[i,'fitness']=testInstance(population,i,dtrain)
        print ("--Fitness %f \n " % population.fitness[i])
    population.sort(['fitness'],ascending=1,inplace=True)
    population.reset_index(drop=True,inplace=True)
    printResult('Result.txt',population,0,run) #print best instances to file
    print("Generation %d Best fitness (5-fold RMSPE CV): %f" %(run, population.fitness[0]))

    

In [ ]:
##############################Testing#########################
i=0 #selecting the best instance
params = {"objective": "reg:linear",
          "eta": population.eta[i],
          "max_depth": population.depth[i],
          "subsample": population.subsample[i],
          "colsample_bytree": population.colsample_bytree[i],
          "num_boost_round":int(population.nRound[i]),
          "lambda":population.lamda[i],
          "alpha":population.alpha[i],
          "gamma":population.gamma[i],
          "min_child_weight":population.min_child_weight[i],
          "silent": 1          
} 
#train the final xgboost model
gbm=xgb.train(params, dtrain,  feval=rmspe_xg, num_boost_round=int(population.nRound[i]), obj=RMSPE_objective, verbose_eval=True)
          
print("Make predictions on the test set")
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
# Make Submission
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv("xgboost_10_submission.csv", index=False)  

# XGBoost + Features

In [ ]:
#!/usr/bin/python
#from __future__ import print_function
'''
Public Score :  0.11771
Private Validation Score : [3886] train-rmspe:0.062407  eval-rmspe:0.088543
'''

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb

# Gather some features
def build_features(features, data):
    # remove NaNs
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
    # Use some properties directly
    features.extend(['Store', 'CompetitionDistance', 'Promo', 'Promo2', 'SchoolHoliday'])

    # Label encode some features
    features.extend(['StoreType', 'Assortment', 'StateHoliday'])
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)

    features.extend(['DayOfWeek', 'Month', 'Day', 'Year', 'WeekOfYear'])
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear

    # CompetionOpen en PromoOpen from https://www.kaggle.com/ananya77041/rossmann-store-sales/randomforestpython/code
    # Calculate time competition open time in months
    features.append('CompetitionOpen')
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    # Promo open time in months
    features.append('PromoOpen')
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)

    # Indicate that sales on that day are in promo interval
    features.append('IsPromoMonth')
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Okt', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1

    return data

## Start of main script

print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(int),
         'PromoInterval': np.dtype(str)}

train = pd.read_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/train_filled_gap.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("/media/EA34F85134F821ED/Python27/Kaggle/Rossmann/store.csv")


print("Assume store open, if not provided")
test.fillna(1, inplace=True)

print("Consider only open stores for training. Closed stores wont count into the score.")
train = train[train["Open"] != 0]
print("Use only Sales bigger then zero")
train = train[train["Sales"] > 0]

print("Join with store")
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

features = []

print("augment features")
train = build_features(features, train)
test = build_features([], test)
print(features)

print('training data processed')

def rmspe(y, yhat):
    return np.sqrt(np.mean(((y - yhat)/y) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y, yhat)

print("Train xgboost model")

params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.1,
          "max_depth": 10,
          "subsample": 0.85,
          "colsample_bytree": 0.4,
          "min_child_weight": 6,
          "silent": 1,
          "thread": 1,
          "seed": 1301
          }
num_boost_round = 1

print("Train a XGBoost model")
X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)
y_train = np.log1p(X_train.Sales)
y_valid = np.log1p(X_valid.Sales)
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=200, \
  feval=rmspe_xg, verbose_eval=True)

print("Validating")
yhat = gbm.predict(xgb.DMatrix(X_valid[features]))
error = rmspe(X_valid.Sales.values, np.expm1(yhat))
print('RMSPE: {:.6f}'.format(error))

print("Make predictions on the test set")
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
# Make Submission
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv("xgboost_39_submission.csv", index=False)

# Function to splite Data & build DL,GBM,DRF,GLM to predict 'Customers'

In [ ]:
# ----------
# 3- Fit a model on train; using test as validation

# Function for doing class test/train/holdout split
def split_fit_predict(data):
    global gbm_cust,drf_cust,glm_cust,dl_cust
    # Classic Test/Train split
    r = data.runif()   # Random UNIForm numbers, one per row
    train = data[r  <= r]
    test  = data[(0.6 <= r) & (r < 0.9)]
    hold  = data[ 0.9 <= r ]
    print "Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow
    x = data.names
    x.remove("Sales")
    x.remove('Customers')

    # Run GBM
    s = time.time()

    gbm_cust = H2OGradientBoostingEstimator(ntrees=500, # 500 works well
                                        max_depth=6,
                                        learn_rate=0.1)


    gbm_cust.train(x=x,
               y="Customers",
               training_frame =train,
               validation_frame=test)

    gbm_elapsed = time.time() - s

    # Run DRF
    s = time.time()

    drf_cust = H2ORandomForestEstimator(ntrees=250, max_depth=30)

    drf_cust.train(x=x,
               y="Customers",
               training_frame =train,
               validation_frame=test)

    drf_elapsed = time.time() - s 


    # Run GLM
    # if "WC1" in bike_names_x: bike_names_x.remove("WC1")
    s = time.time()

    glm_cust = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")

    glm_cust.train(x=x,
               y="Customers",
               training_frame =train,
               validation_frame=test)

    glm_elapsed = time.time() - s

    # Run DL
    s = time.time()

    dl_cust = H2ODeepLearningEstimator(hidden=[50,50,50,50], epochs=50)

    dl_cust.train(x=x,
              y="Customers",
              training_frame=train,
              validation_frame=test)

    dl_elapsed = time.time() - s

    # ----------
    # 4- Score on holdout set & report
    train_r2_gbm = gbm_cust.model_performance(train).r2()
    test_r2_gbm  = gbm_cust.model_performance(test ).r2()
    hold_r2_gbm  = gbm_cust.model_performance(hold ).r2()
    print "GBM R2 TRAIN=",train_r2_gbm,", R2 TEST=",test_r2_gbm,", R2 HOLDOUT=",hold_r2_gbm

    train_r2_drf = drf_cust.model_performance(train).r2()
    test_r2_drf  = drf_cust.model_performance(test ).r2()
    hold_r2_drf  = drf_cust.model_performance(hold ).r2()
    print "DRF R2 TRAIN=",train_r2_drf,", R2 TEST=",test_r2_drf,", R2 HOLDOUT=",hold_r2_drf

    train_r2_glm = glm_cust.model_performance(train).r2()
    test_r2_glm  = glm_cust.model_performance(test ).r2()
    hold_r2_glm  = glm_cust.model_performance(hold ).r2()
    print "GLM R2 TRAIN=",train_r2_glm,", R2 TEST=",test_r2_glm,", R2 HOLDOUT=",hold_r2_glm

    train_r2_dl = dl_cust.model_performance(train).r2()
    test_r2_dl  = dl_cust.model_performance(test ).r2()
    hold_r2_dl  = dl_cust.model_performance(hold ).r2()
    print " DL R2 TRAIN=",train_r2_dl,", R2 TEST=",test_r2_dl,", R2 HOLDOUT=",hold_r2_dl

    # make a pretty HTML table printout of the results

    header = ["Model", "R2 TRAIN", "R2 TEST", "R2 HOLDOUT", 'Test MSE', "Model Training Time (s)"]
    table  = [
              ["GBM", train_r2_gbm, test_r2_gbm, round(hold_r2_gbm), round(gbm_cust.mse()), round(gbm_elapsed,3)],
              ["DRF", train_r2_drf, test_r2_drf, round(hold_r2_drf), round(drf_cust.mse()), round(drf_elapsed,3)],
              ["GLM", train_r2_glm, test_r2_glm, round(hold_r2_glm), round(glm_cust.mse()), round(glm_elapsed,3)],
              ["DL ", train_r2_dl,  test_r2_dl,  round(hold_r2_dl), round(dl_cust.mse()), round(dl_elapsed,3) ],
              ]
    h2o.H2ODisplay(table,header)
    # --------------

In [ ]:
# Split the data (into test & train), fit some models and predict on the holdout data

data = h2o.import_file('/ya/mro/yax90/oemro/Kaggle/train_prep.csv')
split_fit_predict(data)
# Here we see an r^2 of 0.91 for GBM, and 0.71 for GLM.  This means given just
# the station, the month, and the day-of-week we can predict 90% of the
# variance of the bike-trip-starts.

In [ ]:
test = h2o.import_file('/ya/mro/yax90/oemro/Kaggle/test_prep.csv')
testNoID = test.drop('Id')

a = 1
b = 3
c = 0
d = 1
yhat_cust = []
yhat_cust =(a * gbm_cust.predict(testNoID) + b * drf_cust.predict(testNoID) + \
           c * glm_cust.predict(testNoID) + d * dl_cust.predict(testNoID))/sum([a,b,c,d])

In [ ]:
test = test.as_data_frame(use_pandas=True)
yhat_cust = yhat_cust.as_data_frame(use_pandas=True)

In [ ]:
test_cust = pd.concat([test,yhat_cust],axis=1)

In [ ]:
test_cust.columns=['Id', 'Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 
                   'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 
                   'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'year', 
                   'month', 'day', 'Customers']

In [ ]:
test_cust.to_csv('/ya/mro/yax90/oemro/Kaggle/test_cust.csv',index=False)

In [ ]:
test_cust.columns

# Function to splite Data & build DL,GBM,DRF,GLM to predict 'Sales'

In [ ]:
# ----------
# 3- Fit a model on train; using test as validation

# Function for doing class test/train/holdout split
def split_fit_predict(data):
    global gbm_sales,drf_sales,glm_sales,dl_sales
    # Classic Test/Train split
    r = data.runif()   # Random UNIForm numbers, one per row
    train = data[r  <= r]
    test  = data[(0.6 <= r) & (r < 0.9)]
    hold  = data[ 0.9 <= r ]
    print "Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow
    x = data.names
    x.remove('Customers')
    x.remove("Sales")

    # Run GBM
    s = time.time()

    gbm_sales = H2OGradientBoostingEstimator(ntrees=500, # 500 works well
                                        max_depth=6,
                                        learn_rate=0.1)


    gbm_sales.train(x=x,
               y="Sales",
               training_frame =train,
               validation_frame=test)

    gbm_elapsed = time.time() - s

    # Run DRF
    s = time.time()

    drf_sales = H2ORandomForestEstimator(ntrees=500, max_depth=50)

    drf_sales.train(x=x,
                    y="Sales",
                    training_frame =train,
                    validation_frame=test)

    drf_elapsed = time.time() - s 


    # Run GLM
    # if "WC1" in bike_names_x: bike_names_x.remove("WC1")
    s = time.time()

    glm_sales = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")

    glm_sales.train(x=x,
               y="Sales",
               training_frame =train,
               validation_frame=test)

    glm_elapsed = time.time() 
    
    # Run DL
    s = time.time()

    dl_sales = H2ODeepLearningEstimator(hidden=[50,50,50,50], epochs=50)

    dl_sales.train(x=x,
              y="Sales",
              training_frame=train,
              validation_frame=test)

    dl_elapsed = time.time() - s

    # ----------
    # 4- Score on holdout set & report
    train_r2_gbm = gbm_sales.model_performance(train).r2()
    test_r2_gbm  = gbm_sales.model_performance(test ).r2()
    hold_r2_gbm  = gbm_sales.model_performance(hold ).r2()
    print "GBM R2 TRAIN=",train_r2_gbm,", R2 TEST=",test_r2_gbm,", R2 HOLDOUT=",hold_r2_gbm

    train_r2_drf = drf_sales.model_performance(train).r2()
    test_r2_drf  = drf_sales.model_performance(test ).r2()
    hold_r2_drf  = drf_sales.model_performance(hold ).r2()
    print "DRF R2 TRAIN=",train_r2_drf,", R2 TEST=",test_r2_drf,", R2 HOLDOUT=",hold_r2_drf

    train_r2_glm = glm_sales.model_performance(train).r2()
    test_r2_glm  = glm_sales.model_performance(test ).r2()
    hold_r2_glm  = glm_sales.model_performance(hold ).r2()
    print "GLM R2 TRAIN=",train_r2_glm,", R2 TEST=",test_r2_glm,", R2 HOLDOUT=",hold_r2_glm

    train_r2_dl = dl_sales.model_performance(train).r2()
    test_r2_dl  = dl_sales.model_performance(test ).r2()
    hold_r2_dl  = dl_sales.model_performance(hold ).r2()
    print " DL R2 TRAIN=",train_r2_dl,", R2 TEST=",test_r2_dl,", R2 HOLDOUT=",hold_r2_dl

    # make a pretty HTML table printout of the results

    header = ["Model", "R2 TRAIN", "R2 TEST", "R2 HOLDOUT", 'Test MSE', "Model Training Time (s)"]
    table  = [
              ["GBM", train_r2_gbm, test_r2_gbm, round(hold_r2_gbm), gbm_sales.mse(), round(gbm_elapsed,3)],
              ["DRF", train_r2_drf, test_r2_drf, round(hold_r2_drf), drf_sales.mse(), round(drf_elapsed,3)],
              ["GLM", train_r2_glm, test_r2_glm, round(hold_r2_glm), glm_sales.mse(), round(glm_elapsed,3)],
              ["DL ", train_r2_dl,  test_r2_dl,  round(hold_r2_dl), dl_sales.mse(), round(dl_elapsed,3) ],
              ]
    h2o.H2ODisplay(table,header)
    # --------------

In [ ]:
data = h2o.import_file('/media/EA34F85134F821ED/Python27/train_prep.csv')
split_fit_predict(data)

In [ ]:
test = h2o.import_file('/ya/mro/yax90/oemro/Kaggle/test_cust.csv')
testNoID = test.drop('Id')

a = 2
b = 2
c = 0
d = 1
yhat_sales = []
yhat_sales =(a * gbm_sales.predict(testNoID) + b * drf_sales.predict(testNoID) + \
           c * glm_sales.predict(testNoID) + d * dl_sales.predict(testNoID))/sum([a,b,c,d])

In [ ]:
test = test.as_data_frame(use_pandas=True)
yhat_sales = yhat_sales.as_data_frame(use_pandas=True)

In [ ]:
test_sales = pd.concat([test,yhat_sales],axis=1)

In [ ]:
test_sales.columns

In [ ]:
test_sales.columns=['Id', 'Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 
                   'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 
                   'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'year', 
                   'month', 'day', 'Customers','Sales']

In [ ]:
test_sales.to_csv('/ya/mro/yax90/oemro/Kaggle/test_sales.csv',index=False)

In [ ]:
test_sales['Sales'] = np.expm1(test_sales['Sales'])

In [ ]:
test_sales[['Id','Sales']].sort(columns='Id').to_csv('/ya/mro/yax90/oemro/Kaggle/submission.csv',index=False)

# Fit best Params to DRF

In [ ]:
def split_fit_predict_drf(data):
    global drf_sales
    # Classic Test/Train split
    r = data.runif()   # Random UNIForm numbers, one per row
    train = data[r  <= r]
    test  = data[(0.6 <= r) & (r < 0.9)]
    hold  = data[ 0.9 <= r ]
    print "Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow
    x = data.names
    x.remove('Customers')
    x.remove("Sales")

    # Run DRF
    s = time.time()

    drf_sales = H2ORandomForestEstimator(ntrees=500, max_depth=50,balance_classes=True, nbins_cats=1115)

    drf_sales.train(x=x,
                    y="Sales",
                    training_frame =train,
                    validation_frame=test)

    drf_elapsed = time.time() - s 

In [ ]:
data = h2o.import_file('/media/EA34F85134F821ED/Python27/Kaggle/train_prep.csv')
split_fit_predict_drf(data)

## Make Prediction, Transform H2Odf to PDdf, Write Submission

In [ ]:
test = h2o.import_file('/media/EA34F85134F821ED/Python27/Kaggle/test_prep.csv')
testNoID = test.drop('Id')

#gbm_predict = gbm_sales.predict(testNoID)
drf_predict = drf_sales.predict(testNoID)
#glm_predict = glm_sales.predict(testNoID)
#dl_predict = dl_sales.predict(testNoID)

In [ ]:
test = test.as_data_frame(use_pandas=True)
#gbm_predict = gbm_predict.as_data_frame(use_pandas=True)
drf_predict = drf_predict.as_data_frame(use_pandas=True)
#glm_predict = glm_predict.as_data_frame(use_pandas=True)
#dl_predict = dl_predict.as_data_frame(use_pandas=True)
#yhat_sales = yhat_sales.as_data_frame(use_pandas=True)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
ensemble25 = pd.read_csv('D:\Python27\Kaggle\Rossmann/prediction_25_DL_models_avg.csv', index_col=False)

In [ ]:
ensemble25.dtypes

In [ ]:
ensemble25['Avg'] = ensemble25['Avg']

In [ ]:
ensemble25.Avg.head()

In [ ]:
np.expm1(ensemble25['Avg'])

In [ ]:
test = pd.read_csv('D:\Python27\Kaggle\Rossmann/test_prep.csv')

In [ ]:
avg = np.expm1(ensemble25['Avg'])
avg

In [ ]:
ensemble25.columns

In [ ]:
ensemble25['Avg'].head()

In [ ]:
drf = pd.concat([test['Id'],ensemble25['Sales']],axis=1)

In [ ]:
drf.head()

In [ ]:
drf.sort(columns='Id').to_csv('D:\Python27\Kaggle\Rossmann/ensemble_25.csv',index=False)

In [ ]:
#test_sales = pd.concat([test['Id'],yhat_sales],axis=1)
#gbm = pd.concat([test['Id'],gbm_predict],axis=1)
drf = pd.concat([test['Id'],drf_predict],axis=1)
#glm = pd.concat([test['Id'],glm_predict],axis=1)
#dl = pd.concat([test['Id'],dl_predict],axis=1)

#test_sales.columns=['Id', 'Sales']
#gbm.columns=['Id', 'Sales']
drf.columns=['Id', 'Sales']
#glm.columns=['Id', 'Sales']
#dl.columns=['Id', 'Sales']

#test_sales['Sales'] = np.expm1(test_sales['Sales'])
#gbm['Sales'] = np.expm1(gbm['Sales'])
drf['Sales'] = np.expm1(drf['Sales'])
#glm['Sales'] = np.expm1(glm['Sales'])
#dl['Sales'] = np.expm1(dl['Sales'])

#test_sales.sort(columns='Id').to_csv('/ya/mro/yax90/oemro/Kaggle/submission.csv',index=False)
#gbm.sort(columns='Id').to_csv('/media/EA34F85134F821ED/Python27/Kaggle/submission_gbm.csv',index=False)
drf.sort(columns='Id').to_csv('/media/EA34F85134F821ED/Python27/Kaggle/submission_drf.csv',index=False)
#glm.sort(columns='Id').to_csv('/media/EA34F85134F821ED/Python27/Kaggle/submission_glm.csv',index=False)
#dl.sort(columns='Id').to_csv('/media/EA34F85134F821ED/Python27/Kaggle/submission_dl.csv',index=False)

## Root Mean Square Prozent Error from func rmspe 

In [ ]:
rmspe(train.Sales,drf_predict.predict)

In [ ]:
data = h2o.import_file('/media/EA34F85134F821ED/Python27/Kaggle/train_prep.csv')
r = data.runif()
train = data[r  <= r]
test  = data[(0.6 <= r) & (r < 0.9)]
hold  = data[ 0.9 <= r ]
print "Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow
x = data.names
x.remove('Customers')
x.remove("Sales")




drf_predict = drf_sales.predict(train)
drf_predict = drf_predict.as_data_frame(use_pandas=True)
train = train.as_data_frame(use_pandas=True)

rmspe(train.Sales,drf_predict.predict)

# Grid Search DRF

In [ ]:
from h2o.grid.grid_search import H2OGridSearch

data = h2o.import_file('/media/EA34F85134F821ED/Python27/Kaggle/train_prep.csv')

# Extract best Model from Grid Search
def extractBestModel(models):
    bestMse = models[0].mse(xval=True)
    result = models[0]
    for model in models:
        if model.mse(xval=True) < bestMse:
            bestMse = model.mse(xval=True)
            result = model
    return result


# Classic Test/Train split
r = data.runif()   # Random UNIForm numbers, one per row
train = data[r  <= 0.2]
test  = data[(0.6 <= r) & (r < 0.9)]
hold  = data[ 0.9 <= r ]
print "Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow
x = data.names
x.remove("Sales")
x.remove('Customers')

# Run DRF
hyper_parameters = {'ntrees':[10,500], 'max_depth':[100,50,10]}
#,'balance_classes':True, 'nbins_cats':1115}
grid_search = H2OGridSearch(H2ORandomForestEstimator, hyper_params=hyper_parameters)
grid_search.train(x=x, y="Sales", training_frame=train)

                              
bestmodel = extractBestModel(grid_search)
bestmodel

In [ ]:
bestmodel3 = grid_search[5]

In [ ]:
bestmodel3.get_params